# Claude CLI - Les Bases

**Module :** Vibe-Coding / Claude Code / Notebooks CLI  
**Niveau :** Debutant  
**Duree :** 20 min  
**Prerequis :** Claude Code CLI installe, OpenRouter configure

## Objectifs d'Apprentissage

- [ ] Verifier l'installation de Claude CLI
- [ ] Executer une premiere commande `claude -p`
- [ ] Comprendre les formats de sortie (text, json)
- [ ] Selectionner differents modeles (sonnet, opus, haiku)
- [ ] Analyser du code avec Claude

---

## Qu'est-ce que Claude CLI ?

Claude CLI (Command Line Interface) est l'outil en ligne de commande officiel de Claude Code. Il permet d'interagir avec les modeles Claude directement depuis un terminal ou un script, sans passer par une interface graphique.

**Cas d'usage typiques :**
- Automatiser des taches repetitives (generation de code, documentation)
- Integrer Claude dans des pipelines CI/CD
- Creer des scripts de productivite personnalises
- Tester rapidement des prompts sans quitter le terminal

> **Note :** Ce notebook utilise des wrappers Python pour executer les commandes CLI. Vous pouvez aussi executer ces memes commandes directement dans un terminal.

## 1. Configuration et Imports

Commençons par importer les modules necessaires et configurer l'environnement.

In [ ]:
# Configuration
import sys
import os

# Ajouter le repertoire helpers au path
sys.path.insert(0, 'helpers')

# Mode simulation : mettre a True si pas de cle API
SIMULATION_MODE = False

# Import des helpers
try:
    from claude_cli import (
        run_claude, 
        check_claude_status, 
        verify_installation,
        print_response
    )
    # Activer le mode simulation si demande
    import claude_cli
    claude_cli.SIMULATION_MODE = SIMULATION_MODE
    print("Helpers charges avec succes")
except ImportError as e:
    print(f"Erreur d'import: {e}")
    print("Assurez-vous d'etre dans le bon repertoire")

## 2. Verification de l'Installation

Avant de commencer, verifions que Claude CLI est correctement installe.

In [ ]:
# Verifier l'installation
if verify_installation():
    print("Claude CLI est installe")
else:
    print("Claude CLI n'est PAS installe")
    print("Consultez le guide d'installation : INSTALLATION-CLAUDE-CODE.md")

In [ ]:
# Verifier la version
import subprocess

result = subprocess.run(["claude", "--version"], capture_output=True, text=True)
print(f"Version: {result.stdout.strip()}")

In [ ]:
# Verifier le statut de connexion
status = check_claude_status()

if status.get("connected"):
    print("Connexion OK")
    print(f"Modele actif: {status.get('model', 'inconnu')}")
else:
    print(f"Erreur de connexion: {status.get('error', 'inconnue')}")

## 3. Premiere Commande

La commande de base est `claude -p "votre prompt"` qui envoie une question ponctuelle (one-shot).

**Syntaxe :**
```bash
claude -p "Votre question ou instruction"
```

**Explication des parametres :**
- `claude` : L'executable CLI
- `-p` ou `--print` : Mode "print" - pose une question unique et affiche la reponse (pas de conversation)
- Le prompt est entre guillemets pour gerer les espaces

> **Note :** Le mode `-p` est ideal pour des questions ponctuelles. Pour des conversations multi-tours, nous verrons le mode session dans le notebook suivant.

In [ ]:
# Premiere commande simple
stdout, stderr, code = run_claude("Bonjour ! Reponds en une phrase.")

print_response(stdout, stderr, code)

In [ ]:
# Question plus elaboree
stdout, stderr, code = run_claude(
    "Explique en 2 phrases ce qu'est Python."
)

print_response(stdout, stderr, code)

## 4. Formats de Sortie

Claude peut retourner les reponses en differents formats :
- **text** (defaut) : Reponse en texte brut, ideal pour la lecture humaine
- **json** : Reponse structuree en JSON, ideal pour le traitement programmatique

```bash
claude -p "prompt" --output-format json
```

**Quand utiliser JSON ?**
- Pour parser la reponse dans un script
- Pour extraire des donnees structurees (listes, objets)
- Pour integrer Claude dans un pipeline automatise

> **Attention :** Le format JSON ne garantit pas une structure specifique. Incluez des instructions dans votre prompt pour specifier le schema attendu (ex: "retourne un JSON avec les cles 'nom' et 'description'").

In [ ]:
# Format texte (defaut)
stdout, stderr, code = run_claude(
    "Liste 3 langages de programmation populaires",
    output_format="text"
)

print("=== Format Texte ===")
print(stdout)

In [ ]:
# Format JSON
from helpers.claude_cli import run_claude_json

result = run_claude_json(
    "Donne-moi une liste de 3 frameworks web en JSON avec nom et langage"
)

print("=== Format JSON ===")
import json
print(json.dumps(result, indent=2, ensure_ascii=False))

## 5. Selection du Modele

Claude Code propose plusieurs modeles avec differents compromis performance/cout/vitesse :

| Alias | Modele | Vitesse | Intelligence | Cout | Usage recommande |
|-------|--------|---------|--------------|------|------------------|
| `haiku` | Claude Haiku | +++ | + | $ | Questions simples, classification, extraction |
| `sonnet` | Claude Sonnet | ++ | ++ | $$ | Usage quotidien, code, analyse (defaut) |
| `opus` | Claude Opus | + | +++ | $$$ | Raisonnement complexe, preuves, recherche |

**Syntaxe :**
```bash
claude --model opus -p "Question complexe"
claude --model haiku -p "Question simple"
```

> **Tip economique :** Commencez toujours par `haiku` pour vos tests. Passez a `sonnet` ou `opus` uniquement si la qualite est insuffisante. Les couts peuvent varier de 1x (haiku) a 15x (opus) !

In [ ]:
# Test avec Haiku (rapide)
stdout, stderr, code = run_claude(
    "En un mot, quelle est la capitale de la France ?",
    model="haiku"
)

print("=== Modele Haiku (rapide) ===")
print_response(stdout, stderr, code)

In [ ]:
# Test avec Sonnet (equilibre)
stdout, stderr, code = run_claude(
    "Explique la difference entre une liste et un tuple en Python",
    model="sonnet"
)

print("=== Modele Sonnet (equilibre) ===")
print_response(stdout, stderr, code)

## 6. Analyse de Code

Claude excelle dans l'analyse et l'explication de code. Voici comment lui soumettre du code pour analyse :

**Bonnes pratiques :**
1. **Delimitez clairement le code** avec des triple backticks ou une indentation
2. **Specifiez le langage** ("ce code Python", "cette fonction JavaScript")
3. **Posez une question precise** ("identifie les bugs", "explique la ligne 5", "propose une optimisation")

**Structure recommandee du prompt :**
```
[Contexte optionnel]
[Question precise]
[Code a analyser]
```

In [ ]:
# Code a analyser
code_sample = '''
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n-1) + fibonacci(n-2)
'''

# Demander une analyse
prompt = f"Analyse ce code Python et identifie un probleme de performance :\n{code_sample}"

stdout, stderr, code = run_claude(prompt)
print_response(stdout, stderr, code)

In [ ]:
# Demander une amelioration
prompt = f"Propose une version optimisee de cette fonction :\n{code_sample}"

stdout, stderr, code = run_claude(prompt)
print_response(stdout, stderr, code)

## 7. Exercice Pratique

Maintenant c'est a vous ! Completez les cellules suivantes.

In [ ]:
# EXERCICE 1 : Demandez a Claude d'expliquer ce que fait ce code
mystery_code = '''
def f(s):
    return s == s[::-1]
'''

# Indice : Utilisez run_claude() avec un prompt qui inclut le code
# Exemple de structure : "Explique ce que fait ce code Python : {code}"

# Votre code ici
# stdout, stderr, code = run_claude(f"Explique ce que fait ce code Python:\n{mystery_code}")
# print_response(stdout, stderr, code)

In [ ]:
# EXERCICE 2 : Utilisez le modele haiku pour une question simple
# Demandez la syntaxe d'une boucle for en Python

# Indice : Ajoutez le parametre model="haiku" a run_claude()
# Gardez le prompt court et precis pour haiku

# Votre code ici
# stdout, stderr, code = run_claude("Quelle est la syntaxe d'une boucle for en Python ?", model="haiku")
# print_response(stdout, stderr, code)

In [ ]:
# EXERCICE 3 : Demandez une reponse en JSON
# Demandez 5 bonnes pratiques Python avec nom et description

# Indice : Utilisez run_claude_json() et specifiez le format dans le prompt
# Exemple de schema a demander : {"practices": [{"name": "...", "description": "..."}]}

# Votre code ici
# result = run_claude_json(
#     "Liste 5 bonnes pratiques Python. Retourne un JSON avec le format: "
#     '{"practices": [{"name": "nom", "description": "description"}]}'
# )
# import json
# print(json.dumps(result, indent=2, ensure_ascii=False))

## 8. Resume

### Ce que vous avez appris

| Commande | Description | Exemple |
|----------|-------------|---------|
| `claude -p "prompt"` | Question ponctuelle | `claude -p "Bonjour"` |
| `--model <nom>` | Choisir le modele | `--model haiku` |
| `--output-format json` | Sortie JSON | Pour traitement automatise |

### Points cles a retenir

1. **Le flag `-p`** est pour les questions ponctuelles (one-shot), pas les conversations
2. **Haiku** est le plus rapide et economique, **Opus** le plus intelligent
3. **Le format JSON** necessite un prompt bien structure pour obtenir un schema previsible
4. **L'analyse de code** fonctionne mieux avec des questions precises et du code bien delimite

### Commandes utiles

```bash
# Verification
claude --version
claude /status

# Question simple
claude -p "Votre question"

# Avec modele specifique
claude --model haiku -p "Question simple"
claude --model opus -p "Question complexe"

# Sortie JSON
claude -p "Prompt structurant le JSON" --output-format json
```

### Prochaine etape

Dans le notebook suivant, nous verrons comment gerer les **sessions et conversations** pour maintenir un contexte entre plusieurs echanges.

-> [02-Claude-CLI-Sessions.ipynb](02-Claude-CLI-Sessions.ipynb)